# Experiment Dataviz

This notebook has the goal to explore the achieved results through different views </br>
The data source is the aggregated resoults "data/aggregated_results.csv", which compiles the Rouge-L metric for differents groups </br>
The first section are the results reported in the project, the another ones are additional analysis

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio


df = pd.read_csv("data/aggregated_results.csv")

## 1. Report Results (Overview)

In [2]:
## Table plot

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df[col] for col in df.columns],
               fill_color='lavender',
               align='left'))
])

# Show the figure
fig.show()

In [3]:


df["k"] = df["k"].astype("str")
df['color'] = df['model'] + '_' + df['experiment_type']
custom_color_sequence = colors = [
    "#F57F4F",  # Dark Orange
    "#F8B195",  # Light Orange
    "#F6C667",  # Dark Yellow
    "#F8E4A6",  # Light Yellow
    "#6A8EAE",  # Dark Blue
    "#A9D0F5"   # Light Blue
]



fig = px.bar(df[df["task"] == "all"], 
             x='k', 
             y='mean_rougeL', 
             color='color',
             text='mean_rougeL',
             barmode='group',
             title='Mean Rouge-L by Model',
             color_discrete_sequence=custom_color_sequence)

# Customize the layout
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(height=600) 
# Show the figure
fig.show()

In [4]:
df[df["model"] == "llama3"]

,Unnamed: 0,model,k,experiment_type,mean_rougeL,median_rougeL,stdev_rougeL,task,color
16,16,llama3,8,general,0.347239,0.000000,0.446995,all,llama3_general
17,17,llama3,8,inclass,0.685071,1.000000,0.438073,all,llama3_inclass
18,18,llama3,16,general,0.361894,0.000000,0.449894,all,llama3_general
19,19,llama3,16,inclass,0.690117,1.000000,0.437368,all,llama3_inclass
20,20,llama3,32,general,0.367625,0.000000,0.454568,all,llama3_general
...,...,...,...,...,...,...,...,...,...
523,523,llama3,64,inclass,0.516972,0.545455,0.171818,taxonomy_animal,llama3_inclass
524,524,llama3,64,inclass,0.610000,1.000000,0.490207,translation_en-de,llama3_inclass
525,525,llama3,64,inclass,0.706667,1.000000,0.455124,translation_en-es,llama3_inclass
526,526,llama3,64,inclass,0.706667,1.000000,0.449541,translation_en-fr,llama3_inclass


In [5]:
## Comparsion By Task to Llama3 - 64

task_df = df.copy()[(df["task"] != "all") & (df["model"] == "llama3") & (df["k"] == "8")][["task", "experiment_type", "mean_rougeL"]]
pivot_df = task_df.pivot_table(index=['task'], columns='experiment_type', values='mean_rougeL')
diff_df = (pivot_df['inclass']- pivot_df['general']).sort_values(ascending=True)

In [6]:
diff_df

task
active_to_passive          0.002857
rhymes                     0.010000
taxonomy_animal            0.044515
num_to_verbal              0.065238
larger_animal              0.066667
sentiment                  0.103333
sum                        0.110000
sentence_similarity        0.128947
negation                   0.139787
synonyms                   0.150000
word_in_context            0.156667
orthography_starts_with    0.169672
diff                       0.380000
antonyms                   0.400000
second_word_letter         0.453175
translation_en-de          0.570000
translation_en-fr          0.589143
translation_en-es          0.678476
first_word_letter          0.949563
letters_list               0.960437
singular_to_plural         0.966000
dtype: float64

In [7]:
fig = go.Figure()
colors = ['green' if val > 0 else 'red' for val in diff_df]

fig.add_trace(go.Bar(
    x=diff_df.index,
    y=diff_df,
    text=diff_df,
    marker_color=colors  # Set the colors
))

# Update layout for better visualization
fig.update_layout(
    title='Difference in Mean Rouge-L by Task (Inclass - General)',
    xaxis_title='Tasks',
    yaxis_title='Mean Rouge-L Diff',
    template='plotly_white'
)

# Show the plot
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(height=550) 

fig.show()